# Proyecto práctico

## Unidad 3 - Aprendizaje supervisado

El proyecto práctico consiste en abordar un problema de clasificación de documentos textuales. Tenemos a nuestra disposición un dataset de noticias de prensa en español publicada por el medio "CNN Chile".

Las noticias están divididas en 7 categorías temáticas: *'pais','deportes','tendencias','tecnologias','cultura','economia','mundo'*

El proyecto se divide en dos partes:

- Utilizar al menos 3 estrategías para entrenar modelos de clasificación capaces de clasificar las noticias según su categoría temática.

- Explorar cuáles son las características que permiten explicar las decisiones de su modelo.

## 0. Evaluación

El proyecto se realiza de forma individual. Se entrega a más tardar el **lunes 30 de noviembre** en su repositorio GitHub.

**Pauta de evaluación:**

Competencia 1: Aplicar un protocolo de aprendizaje supervisado para resolver un problema clasificación estandar, utilizando un entorno de programación en Python

- < 2 : El protocolo de aprendizaje supervisado utilizado es incompleto y/o presenta errores importantes
- 2 a 3.9 : El protocolo de aprendizaje supervisado utilizado es incompleto o presenta un error importante
- 4 a 5.5 : El protocolo de aprendizaje es completo, no tiene error, pero las estrategias utilizadas son relativamente simples y el rendimiento de los modelos es perfectible.
- 5.6 a 7.0 : El protocolo de aprendizaje es completo, no tiene error y al menos una de las estrategias utilizadas a necesitado un trabajado más avanzado y/o permite obtener un mejor rendimiento.

Competencia 2: Explicar el rendimiento de un modelo de clasificación aplicando un protocolo de evaluación Precision/Recall/F-Score

- < 2 : El trabajo no presenta explicaciones del rendimiento de los modelos de clasificación
- 2 a 3.9 : El trabajo presenta algunas explicaciones pero tienen errores.
- 4 a 5.5 : El trabajo presenta explicaciones correctas del rendimiento de los modelos
- 5.6 a 7 : El trabajo presenta explicaciones correctas del rendimiento de los modelos y además presenta un método para explicar las decisiones/errores


## 1. Dataset

In [77]:
import pandas as pd
from pandasql import sqldf
import spacy

df = pd.read_csv('cnnchile_7000.csv')
df = df.drop(["country","media_outlet", "url","date","title"],1)
df

,text,category
0,La Federación de Estudiantes de la Universidad...,pais
1,La Defensoría de la Niñez emitió este domingo ...,pais
2,El monto del bono es de dos tercios de Unidad ...,pais
3,Una nueva polémica tiene esta carrera presiden...,pais
4,Especialistas recomiendan no consumir más de 2...,pais
...,...,...
6995,Las compañías ya han revelado muchos detalles ...,tecnologias
6996,Se proyecta que tras un virtual empate en 2012...,tecnologias
6997,Tablets y smartphones fueron los regalos tecno...,tecnologias
6998,Crecí jugando clásicos de naves como Terminal ...,tecnologias


In [78]:
q="""SELECT category, count(*) FROM df GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,tendencias,1000
1,tecnologias,1000
2,pais,1000
3,mundo,1000
4,economia,1000
5,deportes,1000
6,cultura,1000


In [79]:
df.shape

(7000, 2)

In [80]:
categories = result["category"].astype("str").tolist()
categories

['tendencias',
 'tecnologias',
 'pais',
 'mundo',
 'economia',
 'deportes',
 'cultura']

In [81]:
q="""SELECT * FROM df WHERE category = "tendencias";"""
df_tend=sqldf(q)

df_tend = df_tend.sample(n=300)

q="""SELECT * FROM df WHERE category = "tecnologias";"""
df_tech = sqldf(q)

df_tech = df_tech.sample(n=300)


q="""SELECT * FROM df WHERE category = "pais";"""
df_pais=sqldf(q)

df_pais = df_pais.sample(n=300)

q="""SELECT * FROM df WHERE category = "mundo";"""
df_mundo=sqldf(q)

df_mundo = df_mundo.sample(n=300)

q="""SELECT * FROM df WHERE category = "economia";"""
df_eco=sqldf(q)

df_eco = df_eco.sample(n=300)

q="""SELECT * FROM df WHERE category = "deportes";"""
df_dep = sqldf(q)

df_dep = df_dep.sample(n=300)

q="""SELECT * FROM df WHERE category = "cultura";"""
df_cult = sqldf(q)

df_cult = df_cult.sample(n=300)

df_train = pd.concat([df_tend, df_tech, df_pais, df_mundo,df_eco,df_dep,df_cult], ignore_index=True)
df_train.shape


(2100, 2)

In [82]:
q="""SELECT category, count(*) FROM df_train GROUP BY category ORDER BY count(*) DESC;"""
test=sqldf(q)
test

,category,count(*)
0,tendencias,300
1,tecnologias,300
2,pais,300
3,mundo,300
4,economia,300
5,deportes,300
6,cultura,300


In [83]:
nlp = spacy.load("es_core_news_md")

### Determinar parametros

In [84]:
## PRUEBAS REALES

docs = df["text"].astype("str").values.tolist()

X = df['text']
ylabels = df['category']

X_train = df_train['text']
X_test = df['text']
y_train = df_train['category']
y_test = df['category']

## MIS PRUEBAS
'''
#docs = df_train["text"].astype("str").values.tolist()

#X = df_train['text']
#ylabels = df_train['category']

#X_train = df_train['text']
#X_test = df_train['text']
#y_train = df_train['category']
#y_test = df_train['category']
'''


### Spacy function

In [85]:
from sklearn.feature_extraction.text import CountVectorizer
from pandas import DataFrame

In [86]:
def feature_extraction(text):
    
    mytokens = nlp(text)

    #Guardamos las palabras como características si corresponden a ciertas categorias gramaticales
    mytokens = [ word for word in mytokens if word.pos_ in ["NOUN", "ADJ", "VERB"] ]
    
    #Transformamos las palabras en minusculas
    mytokens = [ word.lemma_.lower().strip() for word in mytokens ]

    # return preprocessed list of tokens
    return mytokens

In [87]:
bow_vector = CountVectorizer(tokenizer = feature_extraction, min_df=0., max_df=1.0)

### Entreamiento regresión logistica


In [110]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

### Regresión logistica y CountVectorizer

In [89]:
## ENTRENAMIENTO USANDO PIPELINE

model_1 = LogisticRegression()
pipe = Pipeline([('vectorizing', bow_vector),
                 ('learning', model_1)])

# model generation
#Determina los mejores fit para el df usando el modelo_1 y bow_vector
pipe.fit(X_train,y_train)

Pipeline(steps=[('vectorizing',
                 CountVectorizer(min_df=0.0,
                                 tokenizer=<function feature_extraction at 0x7fc9fb7f33b0>)),
                ('learning', LogisticRegression())])

In [90]:
predicted = pipe.predict(X_test) # Vectoriza los datos de test.
#predicted_proba = pipe.predict_proba(X_test) #Vectoriza la probabilidad de ser una de las x posibilidades


In [91]:
# Exactitud del modelo.
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Accuracy: 0.7831428571428571


In [92]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

     cultura       0.91      0.86      0.88      1000
    deportes       0.86      0.86      0.86      1000
    economia       0.78      0.81      0.80      1000
       mundo       0.70      0.76      0.73      1000
        pais       0.76      0.71      0.74      1000
 tecnologias       0.76      0.76      0.76      1000
  tendencias       0.72      0.72      0.72      1000

    accuracy                           0.78      7000
   macro avg       0.78      0.78      0.78      7000
weighted avg       0.78      0.78      0.78      7000



### Regresión logistica y TfidfTransformer

In [93]:
##Usamos los id, en funcion de los poco común se aumentan los pesos de las palabras.

tfidf_vector = TfidfVectorizer(tokenizer = feature_extraction, min_df=0., max_df=1.0)

In [94]:
model_2 = LogisticRegression()

pipe2 = Pipeline([('vectorizing', tfidf_vector),
                 ('learning', model_2)])


In [95]:
pipe2.fit(X_train,y_train)

Pipeline(steps=[('vectorizing',
                 TfidfVectorizer(min_df=0.0,
                                 tokenizer=<function feature_extraction at 0x7fc9fb7f33b0>)),
                ('learning', LogisticRegression())])

In [96]:
predicted = pipe2.predict(X_test)

In [97]:
# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Accuracy: 0.8038571428571428


In [98]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

     cultura       0.87      0.92      0.90      1000
    deportes       0.92      0.88      0.90      1000
    economia       0.83      0.81      0.82      1000
       mundo       0.69      0.78      0.73      1000
        pais       0.78      0.75      0.77      1000
 tecnologias       0.80      0.77      0.78      1000
  tendencias       0.75      0.71      0.73      1000

    accuracy                           0.80      7000
   macro avg       0.81      0.80      0.80      7000
weighted avg       0.81      0.80      0.80      7000



### Regresión logistica y World embedding

In [99]:
### Arreglar la libreria que no me funciona.

### MultinomialNB y TfidfTransformer 

In [104]:
model_3 = MultinomialNB()

pipe3 = Pipeline([('vectorizing', tfidf_vector),
                 ('learning', model_3)])


In [105]:
pipe3.fit(X_train,y_train)

Pipeline(steps=[('vectorizing',
                 TfidfVectorizer(min_df=0.0,
                                 tokenizer=<function feature_extraction at 0x7fc9fb7f33b0>)),
                ('learning', MultinomialNB())])

In [106]:
predicted = pipe3.predict(X_test)

In [107]:
# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Accuracy: 0.7805714285714286


In [108]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

     cultura       0.71      0.97      0.82      1000
    deportes       0.93      0.86      0.89      1000
    economia       0.75      0.87      0.80      1000
       mundo       0.74      0.74      0.74      1000
        pais       0.79      0.74      0.76      1000
 tecnologias       0.76      0.78      0.77      1000
  tendencias       0.87      0.50      0.64      1000

    accuracy                           0.78      7000
   macro avg       0.79      0.78      0.78      7000
weighted avg       0.79      0.78      0.78      7000



### SGDClassifier y TfidfTransformer

In [112]:
model_4 = SGDClassifier(loss='hinge', 
              penalty='l2', 
              alpha=1e-3, 
              random_state=42,
              max_iter=5, 
              tol=None)

In [113]:
pipe4 = Pipeline([('vectorizing', tfidf_vector),
                 ('learning', model_4)])


In [114]:
pipe4.fit(X_train,y_train)

Pipeline(steps=[('vectorizing',
                 TfidfVectorizer(min_df=0.0,
                                 tokenizer=<function feature_extraction at 0x7fc9fb7f33b0>)),
                ('learning',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [115]:
predicted = pipe4.predict(X_test)

In [116]:
# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Accuracy: 0.8012857142857143


In [117]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

     cultura       0.82      0.96      0.88      1000
    deportes       0.88      0.91      0.89      1000
    economia       0.79      0.85      0.82      1000
       mundo       0.70      0.78      0.74      1000
        pais       0.82      0.72      0.77      1000
 tecnologias       0.78      0.79      0.78      1000
  tendencias       0.84      0.62      0.71      1000

    accuracy                           0.80      7000
   macro avg       0.80      0.80      0.80      7000
weighted avg       0.80      0.80      0.80      7000

